In [1]:
from main import main as execute

import os
import pandas as pd
from typing import List, Dict, Tuple

In [2]:
################################################
# INPUT CONFIGURATION- REPLACES OLD .JSON FILE #
################################################

input = {
    "DEVICE_NAME": "CAM3",
    "DEVICE_TYPE": "CAMERA",
    
    "DEVICE_SPECIES": "DIGICAM",
    
    "EXP_SHOT_NOS": [3],
    "BKG_SHOT_NOS": [1],
    
    "BKG_NAME": "DARKFIELD",
    "BACKGROUND_STATUS": "RAW",
    
    "OPERATIONS": {
        
        "PLOT": True,
        
        "AVERAGE_SHOTS":False
        
    }
}

In [3]:
# Parent data path
data_path = '/eos/project/h/hiradmat/HRMT Experiments/2024/FIREBALL2/FB2_HRMT64_repository/HRMT64 data'

# Check to make sure that the parent data path actually exists.
if not os.path.exists(data_path):
    raise NotImplementedError("Error: parent data_path doesn't exist.")

In [10]:
# Path to shot log
shotlog_path = "./example_data/shotlog.csv"

if not os.path.exists(shotlog_path):
    raise NotImplementedError("Error: path to shot log not found.")

In [5]:
# List of subdirectories in parent data path which should
folders = ["SCOPE1", "SCOPE2", "EP_Spectrometer_Cams", "PlasmaCell_Cams"]

In [6]:
# Make sure all of the folders actually exist.
for folder in folders:
    new_path = os.path.join(data_path, folder)
    if not os.path.exists(new_path):
        raise NotImplementedError(f"Path {new_path} to folder {folder} does not exist.\n")

In [7]:
# Template file names at which the data will be stored

template_file_names = {
    "DIGICAM": 'HRM{}.DigiCam_OD{}_{}_{}.csv',
    "PROBE": 'C{}_SCOPE{}_{}.csv'
}

In [12]:
template = "Hello, {}- I am {}"
template.format("John", "Hayden")

'Hello, John- I am Hayden'

In [11]:
def get_HRM_datapaths_dict(shot_nos:List[int], 
                           cam_no:int, 
                           gain:int,
                           parent_directory:str,
                           template_fname='HRM{}.DigiCam_OD{}_{}_{}.csv', 
                           shotlog_path:str = shotlog_path
                          ):
    
    """Creates a dictionary of data paths for the HRMX camera, indexed by shot number- i.e. a dictionary of form
    {SHOT NO : "/path/"}. This is done when the user supplies a list of shot numbers which they're interested in, as well
    as the gain/camera number.
    
    Parameters
    ----------
        shot_nos : List[int]
            List of integer shot numbers whose data we are interested in.
        cam_no
        parent_directory
        template_fname
        shotlog_path : str
            Path to the shotlog _AS A .CSV_ !!!
    """

    # Check that the path exists
    if not os.path.exists(shotlog_path):
        raise NotImplementedError(f"Shotlog path {shotlog_path} doesn\'t exist.")
    # Check that the path to the shotlog ends in a .csv
    if not shotlog_path.endswith('.csv'):
        raise ValueError(f"Error: shotlog_path should end in .csv, but path provided was {shotlog_path}")

    # Initialize empty data path dictionary
    data_path_dict = {}

    # Open the .csv as a pandas dataframe
    df = pd.read_csv(shotlog_path, delimiter=',')

    #Iterate through the dataframe looking for data corresponding to the specified shot numbers.
    for i, no in df["Shot number"]:
        if no in shot_nos:
            timestamp = str(df["Global UNIX Timestamp (UTC timezone) of HiRadMat cycle"][i])
            acq_cycle = str(df["Acquisition UNIX Timestamp of HiRadMat cycle (UTC timezone)"][i])

            # INITIALIZE THE NAME OF THE FILE
            fname = template_fname.format(cam_no, gain, timestamp, acq_cycle)

            # CREATE THE NAME OF THE PATH
            path = os.path.join(parent_directory, fname)
            
            data_path_dict[no] = path # add the appropriate data path to the data path dictionary
            # using the shot number as the key.
        
    # Return the path dictionary
    return data_path_dict

In [ ]:
def get_scope_datapaths_dict(shot_nos:List[int],
                             scope_no:str,
                             parent_dir:str,
                             template_fname="C{}_SCOPE{}_{}.csv"
                             shotlog_path:str=shotlog_path
):
"""Creates as 